In [1]:
#  引入数据库数据
import duckdb
dbPath = "../data01"  # 数据地址jupytert数据跨级
c = duckdb.connect(dbPath, read_only=False)
l = c.cursor()

In [4]:
l.sql("show tables").df()

,name
0,listing_his
1,order_his
2,product


In [5]:
df = l.sql("from listing_his where account_id = 83").df()
df.to_excel("res.xlsx",index=False)

In [2]:
l.sql("select * from listing_his").show()

┌────────────┬──────────────────────────────┬───────────────┬─────────────────────┐
│ account_id │          seller_sku          │      sku      │      open_date      │
│   int32    │           varchar            │    varchar    │      timestamp      │
├────────────┼──────────────────────────────┼───────────────┼─────────────────────┤
│         81 │ zjch_dxc1412230120711_34cbd  │ 1412230120711 │ 2023-08-21 11:46:01 │
│         81 │ zjch_dxc1412230137411_6edf1  │ 1412230137411 │ 2023-08-21 11:46:01 │
│         81 │ zjch_qjq1411230266611_06958  │ 1411230266611 │ 2023-08-21 19:28:06 │
│         81 │ zjch_qjq1411230267511_c05e0  │ 1411230267511 │ 2023-08-21 19:28:06 │
│         81 │ zjch_qjq1411230286011_2bedb  │ 1411230286011 │ 2023-08-21 19:28:06 │
│         81 │ zjch_qjq14142300356_5111b-11 │ 1414230035611 │ 2023-08-21 19:26:59 │
│         81 │ zjch_qjq14142300356_5111b-12 │ 1414230035612 │ 2023-08-21 19:26:59 │
│         81 │ zjch_qjq14142300356_5111b-13 │ 1414230035613 │ 2023-08-21 19:

In [3]:
l.sql("from order_his").show()

┌────────────────┬────────────┬──────────────────────┬───────────────┬───┬──────────┬──────────┬─────────────┐
│    order_id    │ account_id │      seller_sku      │      sku      │ … │ paymonth │ currency │ total_price │
│    varchar     │   int32    │       varchar        │    varchar    │   │ varchar  │ varchar  │   double    │
├────────────────┼────────────┼──────────────────────┼───────────────┼───┼──────────┼──────────┼─────────────┤
│ AM220803514185 │        211 │ xuuy-WJJM22739-59c63 │ JM22739       │ … │ 202208   │ CAD      │       35.66 │
│ AM220803506579 │         54 │ AAC-LYT-TJ03744-TY…  │ TJ03744-02    │ … │ 202208   │ CAD      │       15.73 │
│ AM220803537712 │       1248 │ WT-Rote-14837-1412…  │ 1412210132411 │ … │ 202208   │ USD      │       11.17 │
│ AM220803543092 │       1248 │ WT-Rote-14837-1412…  │ 1412210132411 │ … │ 202208   │ USD      │       11.17 │
│ AM220803534972 │       1719 │ 1cdSJ01237-ZLM-Wan…  │ SJ01237-01    │ … │ 202208   │ USD      │        9.92 │
│

In [ ]:
l.sql("select distinct line1 from product").df()['line1']

0             健康美容
1             消费电子
2             家居园艺
3             仪器仪表
4             影视摄影
5             乐器用品
6             工业用品
7             户外用品
8             宠物用品
9             服装服饰
10            智能安防
11            包装材料
12    家居（kevin组专用）
13            手机周边
14            工艺收藏
15            手表首饰
16                
17            汽摩用品
18            电子用品
19            母婴用品
20            电脑用品
Name: line1, dtype: object

In [7]:
# 数据运算
from tools.config import *
engine25 = create_engine(loc_con)

 # 根据产品线数据分布计算
for i in ['健康美容','消费电子','家居园艺','仪器仪表','影视摄影','乐器用品','工业用品','户外用品','宠物用品','服装服饰','智能安防','手机周边','工艺收藏','汽摩用品','电子用品','母婴用品','电脑用品']:
# for i in ['健康美容']:

  sql_m1 = f"""
  with
  l1 as (select sku from product where line1 ='{i}'),
  listing1 as (select * from listing_his where sku in (select sku from l1)),
  order1 as (select * from order_his where account_id ||seller_sku in (select account_id || seller_sku from listing1)),
  res_listing as (
      select 
      '{i}' as line,
      a.account_id,
      a.open_date,
      a.sku, 
      a.seller_sku, 
      b.payment_time,
      b.total_price,
      b.currency,
      c.rate,
      date_diff('day', a.open_date ,b.payment_time) as order_day,
      from listing1 a 
      left join order1 b
      on a.account_id = b.account_id and a.seller_sku = b.seller_sku
      left join currency c
      on c.currency = b.currency and b.paymonth = c.paymonth
  ),
  -- 出单数据分类
  order_lsiting as (
  select  
  '{i}' as line,
  count(account_id || seller_sku) as order_listing_num,
  count(distinct sku) as order_sku_num,
  sum( round((total_price * rate ), 2)) as sale_rmb,
  CASE
      WHEN order_day < 30 THEN '<30'
      WHEN order_day <= 60 THEN '30~60'
      WHEN order_day <= 90 THEN '60~90'
      WHEN order_day <= 120 THEN '90~120'
      WHEN order_day <= 150 THEN '120~150'
      WHEN order_day <= 180 THEN '150~180'
      WHEN order_day <= 210 THEN '180~210'
      WHEN order_day <= 240 THEN '210~240'
      WHEN order_day <= 270 THEN '240~270'
      WHEN order_day <= 300 THEN '270~300'
      WHEN order_day <= 330 THEN '300~330'
      WHEN order_day <= 360 THEN '330~360'
      ELSE '360+'
  END AS order_day_mark
  from res_listing where order_day is not null 
  group by order_day_mark, line
  ),
  --原始的数据
  p1 as (
    pivot res_listing USING count(account_id || seller_sku) as listing_num,count(distinct sku) as sku_num group by line
  )
  select * from order_lsiting a
  left join p1 b on a.line = b.line
  """
  # l.sql(sql_m1).show()

  # 产品线数量动销测算
  #   select * from order_lsiting a
  # left join p1 b on a.line = b.line

  df = l.sql(sql_m1).df()
  df['line1'] = i
  print(df)
  df.to_sql("res_rate_his_with_salermb",engine25, index=False, if_exists='append')

    line  order_listing_num  order_sku_num    sale_rmb order_day_mark line_2  \
0   健康美容              64710          14215  8272023.98          30~60   健康美容   
1   健康美容              10332           3195  1362108.59           360+   健康美容   
2   健康美容              14231           4518  1794602.72        270~300   健康美容   
3   健康美容              32069           8002  4082806.91        180~210   健康美容   
4   健康美容              23361           6519  2985959.08        210~240   健康美容   
5   健康美容              61118          13608  7669999.56            <30   健康美容   
6   健康美容              11109           3833  1407473.59        300~330   健康美容   
7   健康美容              43841          10081  5668517.82        120~150   健康美容   
8   健康美容              17678           5445  2237782.36        240~270   健康美容   
9   健康美容              55793          11895  7053646.40          60~90   健康美容   
10  健康美容               8583           3079  1114343.35        330~360   健康美容   
11  健康美容              49929          108

In [ ]:
# 创建表
sql_create = f"""
create table order_his (
order_id varchar,
account_id int,
seller_sku varchar,
sku varchar,
payment_time datetime,
paymonth varchar,
currency varchar,
total_price double
)
"""
# l.execute(sql_create)
# print("create successful")
# l.execute("ALTER TABLE order_his ALTER total_price TYPE DOUBLE")

In [ ]:
# l.execute("drop table order_his")
# l.sql('select count(1) from order_his')
# l.execute("truncate order_his")

In [ ]:
# c.sql("truncate listing_his")
# c.sql("from listing_his")

In [ ]:
print("关闭数据库")
c.close()

关闭数据库
